## Installation script

In [43]:
pip install "/Users/brianburns/Desktop/rivermod/river"

Processing /Users/brianburns/Desktop/rivermod/river
  Created wheel for river: filename=river-1-cp39-cp39-macosx_10_9_x86_64.whl size=1196002 sha256=861de99ea9c300d1c89beb43b097ced9af5a290509c2f8d5a7118907254effda
  Stored in directory: /private/var/folders/s0/cs0fw3px6tx5srp431pdvjc80000gn/T/pip-ephem-wheel-cache-ip7l4tqk/wheels/f8/d0/fa/b4509dd1552e6eb06e093be8298ad615976c630218856e3b35
Successfully built river
  Attempting uninstall: river
    Found existing installation: river 1
    Uninstalling river-1:
      Successfully uninstalled river-1
You should consider upgrading via the '/Users/brianburns/ml/my_env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from river import stream
import math
import matplotlib.pyplot as plt

from river import datasets
from river import evaluate
from river import metrics
from river import preprocessing  # we are going to use that later
from river import synth  # we are going to use some synthetic datasets too
from river import tree

from queue import Queue

import os
from datasketches import kll_floats_sketch
from river.tree.nodes.branch import DTBranch

from river.ensemble import AdaptiveRandomForestRegressor
from river.ensemble import AdaptiveRandomForestRegressorCP
from river.ensemble import AdaptiveRandomForestRegressorQRF

## Load data

In [2]:
def get_target_variable(file_path):
    """
    Our data files have different names for the target variable.
    This returns the string name of the target variable of a dataset in an arff file.
    """
    data_stream = stream.iter_arff(file_path)
    v = next(iter(data_stream))
    return list(v[0].keys())[-1]

In [3]:
# # Demo the above function
# for file_name in os.listdir("data/stationary"):
#     if file_name[0] != ".":
#         file_path = "data/stationary/"+file_name
#         print("file name: "+ file_name, ", target: " + get_target_variable(file_path))

In [4]:
def load_datastream(arff_file):
    target_name = get_target_variable(arff_file)
    datastream = stream.iter_arff(arff_file, target = target_name)
    return datastream

In [5]:
def prequential_mean_error_rate(model, datastream, alpha):
    n_seen = 0
    errors = 0
    for x,y in datastream:
        interval = model.predict_interval(x, alpha)
        if y < interval[0] or y > interval[1]:
            errors += 1
        n_seen += 1
        model.learn_one(x,y)
    return errors/n_seen

## test area

In [28]:
prequential_mean_error_rate(model, ds_list[0:2500],0.2)

0.1452

In [32]:
ds = load_datastream("data/stationary/2dplanes.arff")
ds_list = list(ds)

In [9]:
x,y = ds_list[28]

In [10]:
x,y = ds_list[29]

In [11]:
print(x,y)

{'x1': -1.0, 'x2': -1.0, 'x3': 0.0, 'x4': -1.0, 'x5': 1.0, 'x6': -1.0, 'x7': 1.0, 'x8': 0.0, 'x9': -1.0, 'x10': -1.0} -1.13159


In [7]:
model = AdaptiveRandomForestRegressorQRF(n_models=10, lambda_value = 1)

In [13]:
model.predict_one(x)

0.0

In [41]:
model.predict_interval(x, 0.4)

[-1.1315900087356567, 10.0]

In [31]:
model.k_sketch

200

In [29]:
alpha = 0.1
final_sketch = kll_floats_sketch(model.k_sketch)
for m in model.models:
    if m.model._root is not None:
        if isinstance(m.model._root, DTBranch):
            leaf = m.model._root.traverse(x, until_leaf=True)
        else:
            leaf = m.model._root
        if hasattr(leaf, "kll_sketch"):
            final_sketch.merge(leaf.kll_sketch)

In [30]:
print(final_sketch)

### KLL sketch summary:
   K              : 200
   min K          : 200
   M              : 8
   N              : 217
   Epsilon        : 1.33%
   Epsilon PMF    : 1.65%
   Empty          : false
   Estimation mode: true
   Levels         : 2
   Sorted         : false
   Capacity items : 333
   Retained items : 117
   Min value      : -7.35
   Max value      : 4.6
### End sketch summary



In [25]:
model.learn_one(x,y)

[ForestMemberRegressor (
  index_original=0
  model=BaseTreeRegressor (
    max_features=3
    grace_period=50
    max_depth=inf
    split_confidence=0.01
    tie_threshold=0.05
    leaf_prediction="model"
    leaf_model=LinearRegression (
      optimizer=SGD (
        lr=Constant (
          learning_rate=0.01
        )
      )
      loss=Squared ()
      l2=0.
      l1=0.
      intercept_init=0.
      intercept_lr=Constant (
        learning_rate=0.01
      )
      clip_gradient=1e+12
      initializer=Zeros ()
    )
    model_selector_decay=0.95
    nominal_attributes=None
    splitter=EBSTSplitter ()
    min_samples_split=5
    binary_split=False
    max_size=500
    memory_estimate_period=2000000
    stop_mem_management=False
    remove_poor_attrs=False
    merit_preprune=True
    seed=893565290075911
  )
  created_on=1
  drift_detector=ADWIN (
    delta=0.001
  )
  warning_detector=ADWIN (
    delta=0.01
  )
  is_background_learner=False
  metric=MSE ()
), ForestMemberRegressor (

In [31]:
model.learn_two(x,10)

[ForestMemberRegressor (
  index_original=0
  model=BaseTreeRegressor (
    max_features=3
    grace_period=50
    max_depth=inf
    split_confidence=0.01
    tie_threshold=0.05
    leaf_prediction="model"
    leaf_model=LinearRegression (
      optimizer=SGD (
        lr=Constant (
          learning_rate=0.01
        )
      )
      loss=Squared ()
      l2=0.
      l1=0.
      intercept_init=0.
      intercept_lr=Constant (
        learning_rate=0.01
      )
      clip_gradient=1e+12
      initializer=Zeros ()
    )
    model_selector_decay=0.95
    nominal_attributes=None
    splitter=EBSTSplitter ()
    min_samples_split=5
    binary_split=False
    max_size=500
    memory_estimate_period=2000000
    stop_mem_management=False
    remove_poor_attrs=False
    merit_preprune=True
    seed=736347544120600
  )
  created_on=0
  drift_detector=ADWIN (
    delta=0.001
  )
  warning_detector=ADWIN (
    delta=0.01
  )
  is_background_learner=False
  metric=MSE ()
), ForestMemberRegressor (

In [28]:
for m in model:
    print(m)
    if m.model._root is not None:
        print("F")

ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F
ForestMemberRegressor
F


In [45]:
model.models[0].model._root

Mean: -298.223288 | Var: 234,526.528849

In [46]:
model = AdaptiveRandomForestRegressorCP(n_models=10)
i = 0

for x,y in data_stream:
    model.learn_one(x, y)

model

NameError: name 'data_stream' is not defined

In [14]:
ds_list = list(ds)

In [15]:
ds_list[0]

({'x1': -1.0,
  'x2': 0.0,
  'x3': -1.0,
  'x4': -1.0,
  'x5': 1.0,
  'x6': 1.0,
  'x7': 1.0,
  'x8': 0.0,
  'x9': -1.0,
  'x10': 0.0},
 3.95676)

In [20]:
ds_list[2]

({'x1': 1.0,
  'x2': 0.0,
  'x3': 1.0,
  'x4': 1.0,
  'x5': -1.0,
  'x6': -1.0,
  'x7': 0.0,
  'x8': 1.0,
  'x9': 1.0,
  'x10': 1.0},
 4.71592)

In [31]:
tree_model = model.models[0]

In [32]:
model.l_oob_add(x,y,tree_model)

In [35]:
len(model.l_oob)

28

In [36]:
model.l_oob

{("{'x1': 1.0, 'x2': 0.0, 'x3': -1.0, 'x4': 1.0, 'x5': 1.0, 'x6': 1.0, 'x7': 0.0, 'x8': 1.0, 'x9': 1.0, 'x10': 0.0}",
  '2.67303'): [ForestMemberRegressor (
    index_original=0
    model=BaseTreeRegressor (
      max_features=3
      grace_period=50
      max_depth=inf
      split_confidence=0.01
      tie_threshold=0.05
      leaf_prediction="model"
      leaf_model=LinearRegression (
        optimizer=SGD (
          lr=Constant (
            learning_rate=0.01
          )
        )
        loss=Squared ()
        l2=0.
        l1=0.
        intercept_init=0.
        intercept_lr=Constant (
          learning_rate=0.01
        )
        clip_gradient=1e+12
        initializer=Zeros ()
      )
      model_selector_decay=0.95
      nominal_attributes=None
      splitter=EBSTSplitter ()
      min_samples_split=5
      binary_split=False
      max_size=500
      memory_estimate_period=2000000
      stop_mem_management=False
      remove_poor_attrs=False
      merit_preprune=True
      s

In [37]:
x_s, y_s = repr(x), repr(y)

In [38]:
x_s

"{'x1': 1.0, 'x2': -1.0, 'x3': 0.0, 'x4': 0.0, 'x5': 1.0, 'x6': -1.0, 'x7': 1.0, 'x8': -1.0, 'x9': 0.0, 'x10': 1.0}"

In [39]:
(x_s, y_s) in model.l_oob

True

In [44]:
model.l_oob[(x_s,y_s)].append(10)

In [41]:
lll = []

In [46]:
lll.remove(10)

ValueError: list.remove(x): x not in list

In [43]:
lll

[1]

In [45]:
model.l_oob[(x_s,y_s)]

[ForestMemberRegressor (
   index_original=0
   model=BaseTreeRegressor (
     max_features=3
     grace_period=50
     max_depth=inf
     split_confidence=0.01
     tie_threshold=0.05
     leaf_prediction="model"
     leaf_model=LinearRegression (
       optimizer=SGD (
         lr=Constant (
           learning_rate=0.01
         )
       )
       loss=Squared ()
       l2=0.
       l1=0.
       intercept_init=0.
       intercept_lr=Constant (
         learning_rate=0.01
       )
       clip_gradient=1e+12
       initializer=Zeros ()
     )
     model_selector_decay=0.95
     nominal_attributes=None
     splitter=EBSTSplitter ()
     min_samples_split=5
     binary_split=False
     max_size=500
     memory_estimate_period=2000000
     stop_mem_management=False
     remove_poor_attrs=False
     merit_preprune=True
     seed=268385505019275
   )
   created_on=1
   drift_detector=ADWIN (
     delta=0.001
   )
   warning_detector=ADWIN (
     delta=0.01
   )
   is_background_learner=Fals

In [48]:
model.l_oob[(x_s,y_s)].remove(10)

In [29]:
list(model.calibration_set)

TypeError: 'Queue' object is not iterable

In [18]:
q = Queue(maxsize = 100)

In [19]:
q.maxsize

100

In [2]:
def learn_two(self, x: dict, y: base.typing.Target, **kwargs):
        self._n_samples_seen += 1

        if not self:
            self._init_ensemble(list(x.keys()))

        for model in self.models:
            # Get prediction for instance
            y_pred = model.predict_one(x)

            # Update performance evaluator
            model.metric.update(y_true=y, y_pred=y_pred)

            k = poisson(rate=self.lambda_value, rng=self._rng)
            if k > 0:
                model.learn_one(
                    x=x, y=y, sample_weight=k, n_samples_seen=self._n_samples_seen
                )

        return self

NameError: name 'base' is not defined